In [1]:
# Import packages from the Python standard library
import importlib.util
import os
import sys
import pprint
import time
import warnings
from pathlib import Path


def register_python_source_file(module_name: str, filepath: Path):
    """Import a source file directly.

    Args:
        module_name: The module name to associate with the imported source file.
        filepath: The path to the source file.

    Notes:
        Adapted from the following implementation in the Python documentation:
        https://docs.python.org/3/library/importlib.html#importing-a-source-file-directly
    """
    spec = importlib.util.spec_from_file_location(module_name, str(filepath))
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)


# Filter out warning messages
warnings.filterwarnings("ignore")

# Experiment name
EXPERIMENT_NAME = "mnist"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = "http://localhost:20080"

# Set DIOPTRA_RESTAPI_URI variable if not defined, used to connect to RESTful API service
os.environ["DIOPTRA_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = "http://localhost:35000"

# Set MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
if os.getenv("MLFLOW_TRACKING_URI") is None:
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Register the examples/scripts directory as a Python module
register_python_source_file("scripts", Path("..", "scripts", "__init__.py"))

#from scripts.client import DioptraClient
from scripts.utils import make_tar

# Import third-party Python packages
import numpy as np
from mlflow.tracking import MlflowClient

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

In [2]:
from __future__ import annotations

import os
from pathlib import Path
from posixpath import join as urljoin
from typing import Any
from urllib.parse import urlparse, urlunparse
import inspect
import requests
from typing import Any, cast

import structlog
from structlog.stdlib import BoundLogger

LOGGER: BoundLogger = structlog.stdlib.get_logger()

class APIConnectionError(Exception):
    '''Class for connection errors'''
class StatusCodeError(Exception):
    '''Class for status code errors'''
class JSONDecodeError(Exception):
    '''Class for JSON decode errors'''

def create_data_dict(**kwargs):
    return kwargs

def debug_request(url, method, data=None):
    LOGGER.debug("Request made.", url=url, method=method, data=data)
def debug_response(json):
    LOGGER.debug("Response received.", json=json)

def get(session, endpoint, *features):
    debug_request(urljoin(endpoint, *features), 'GET')
    return make_request(session, 'get', endpoint, None, *features)
def post(session, endpoint, data, *features):
    debug_request(urljoin(endpoint, *features), 'POST', data)
    return make_request(session, 'post', endpoint, data, *features)
def delete(session, endpoint, data, *features):
    debug_request(urljoin(endpoint, *features), 'DELETE', data)
    return make_request(session, 'delete', endpoint, data, *features)
def put(session, endpoint, data, *features):
    debug_request(urljoin(endpoint, *features), 'PUT', data)
    return make_request(session, 'put', endpoint, data, *features)

def make_request(session, method_name, endpoint, data, *features):
    url = urljoin(endpoint, *features)
    method = getattr(session, method_name)
    try: 
        if (data):
            response = method(url, json=data)
        else:
            response = method(url)
        if response.status_code != 200:
            raise StatusCodeError() 
        json = response.json()
    except (requests.ConnectionError, StatusCodeError, requests.JSONDecodeError) as e:
        handle_error(session, url, method_name.upper(), data, response, e) 
    debug_response(json=json)
    return json

def handle_error(session, url, method, data, response, error):
    if (type(error) is requests.ConnectionError):
        restapi = os.environ["DIOPTRA_RESTAPI_URI"]
        message = f"Could not connect to the REST API. Is the server running at {restapi}?"
        LOGGER.error(message, url=url, method=method, data=data, response=response.text);
        raise APIConnectionError(message);
    if (type(error) is StatusCodeError):
        message = f"Error code {response.status_code} returned."
        LOGGER.error(message, url=url, method=method, data=data, response=response.text);
        raise StatusCodeError(message);
    if (type(error) is requests.JSONDecodeError):
        message = f"JSON response could not be decoded."
        LOGGER.error(message, url=url, method=method, data=data, response=response.text);
        raise JSONDecodeError(message);

        

class DioptraClient(object):
    def __init__(self, session=None, address=None, api_version="v1"):
        address = (
            f"{address}/api/{api_version}"
            if address
            else f"{os.environ['DIOPTRA_RESTAPI_URI']}/api/{api_version}"
        )

        self._session = session if session is not None else requests.Session()
        self._users = UsersClient(session, "users", address)
        self._auth = AuthClient(session, "auth", address)
        self._queues = QueuesClient(session, "queues", address)
        self._groups = GroupsClient(session, "groups", address)
        self._tags = TagsClient(session, "tags", address)
        self._plugins = PluginsClient(session, "plugins", address)
        self._pluginParameterTypes = PluginParameterTypesClient(session, "pluginParameterTypes", address)
        self._experiments = ExperimentsClient(session, "experiments", address)
        self._jobs = JobsClient(session, "jobs", address)
        self._entrypoints = EntrypointsClient(session, "entrypoints", address)
        self._models = ModelsClient(session, "models", address)
        self._artifacts = ArtifactsClient(session, "artifacts", address)
        # models
        # artifacts
    @property
    def users(self):
        return self.get_endpoint(self._users)
    @property
    def auth(self):
        return self.get_endpoint(self._auth)
    @property
    def queues(self):
        return self.get_endpoint(self._queues)
    @property
    def groups(self):
        return self.get_endpoint(self._groups)
    @property
    def tags(self):
        return self.get_endpoint(self._tags)
    @property
    def plugins(self):
        return self.get_endpoint(self._plugins)
    @property
    def pluginParameterTypes(self):
        return self.get_endpoint(self._pluginParameterTypes)
    @property
    def experiments(self):
        return self.get_endpoint(self._experiments)
    @property
    def jobs(self):
        return self.get_endpoint(self._jobs)
    @property
    def entrypoints(self):
        return self.get_endpoint(self._entrypoints)
    @property
    def models(self):
        return self.get_endpoint(self._models)
    @property
    def artifacts(self):
        return self.get_endpoint(self._artifacts)

    
    def get_endpoint(self, ep):
        ep.session = self._session
        return ep

class HasTagsProvider(object):
    def __init__(self, url, session):
        self._tags = TagsProvider(url, session)
    @property
    def tags(self):
        return self.get_endpoint(self._tags)
        
    def get_endpoint(self, ep):
        ep.session = self._session
        return ep

class HasDraftsEndpoint(object):
    def __init__(self, url, session, address, fields, put_fields=None):
        self.draft_fields = fields
        self.put_fields = put_fields if put_fields is not None else fields
        self._drafts = DraftsEndpoint(url, self, session, 'draft', address)
    @property
    def drafts(self):
        return self.get_endpoint(self._drafts)
        
    def get_endpoint(self, ep):
        ep.session = self._session
        return ep

class HasSubEndpointProvider(object):
    def __init__(self, url):
        self._url = url
    def idurl(self, ep_id):
        return urljoin(self._url, ep_id)

            
class Endpoint(object):
    def __init__(self, session, ep_name, address):
        self._scheme, self._netloc, self._path, _, _, _ = urlparse(address)
        self._ep_name = ep_name
        self._session = session
    @property
    def session(self):
        return self._session
    @session.setter
    def session(self, s):
        self._session = s
    @property
    def url(self):
        return self.def_endpoint(self._ep_name)
    @property
    def ep_name(self):
        return self._ep_name
    def def_endpoint(self, name):
        '''creates base url for an endpoint by name'''
        return urlunparse(
            (self._scheme, self._netloc, urljoin(self._path, name + "/"), "", "", "")
        )
class SubEndpoint(Endpoint):
    def __init__(self, parent, session, ep_name, address):
        Endpoint.__init__(self, session, ep_name, address)
        self._parent = parent # parent should extend HasSubEndpointProvider
    def suburl(self, ep_id):
        return urljoin(self._parent.idurl(str(ep_id)), self.ep_name)

class UsersClient(Endpoint):
    def get_all(self):
        '''gets all users'''
        return get(self.session, self.url)
    def create(self, username, email, password, confirm_password):
        '''creates a user'''
        d = {"username":username,
             "email": email,
             "password": password,
             "confirmPassword": confirm_password}
        return post(self.session, self.url, d)
    def get_by_id(self, user_id):
        '''get a user by id'''
        return get(self.session, self.url, str(user_id))
    def update_password_by_id(self, user_id, old_password, new_password, confirm_new_password):
        '''change a user's password by id'''
        d = {"oldPassword":old_password,
             "newPassword": new_password,
             "confirmNewPassword": confirm_new_password}
        return post(self.session, self.url, d, str(user_id), 'password')
    def current(self):
        '''get the current user'''
        return get(self.session, self.url, 'current')
    def delete_current(self, password):
        '''delete the current user'''
        d = {"password":password}
        return delete(self.session, self.url, d, 'current')
    def modify_current(self, username, email):
        '''modify the current user'''
        d = {"username":username,
             "email": email}
        return put(self.session, self.url, d, 'current')
    def modify_current_password(self, old_password, new_password, confirm_new_password):
        '''modify the current user's password'''
        d = {"oldPassword":old_password,
             "newPassword": new_password,
             "confirmNewPassword": confirm_new_password}
        return post(self.session, self.url, d, 'current', 'password')
    def failed_user_post(self):
        '''create a post request with an invalid schema, for testing'''
        return post(self.session, self.url, {'a':'doesnotexist'})
    def failed_user_get(self):
        '''create a get request to an invalid url, for testing'''
        return get(self.session, self.url, 'doesnotexist')

class AuthClient(Endpoint):
    def login(self, username, password):
        '''login as the given user'''
        d = {"username":username,
             "password": password}
        return post(self.session, self.url, d, 'login')
    def logout(self, everywhere):
        '''logout as the current user'''
        d = {"everywhere": everywhere}
        return post(self.session, self.url, d, 'logout')

class GroupsClient(Endpoint):
    def get_all(self):
        '''get all groups'''
        return get(self.session, self.url)
    def get_by_id(self, gid):
        '''get a group by id'''
        return get(self.session, self.url, str(gid))
        
class QueuesClient(Endpoint, HasDraftsEndpoint, HasSubEndpointProvider):
    def __init__(self, session, ep_name, address):
        Endpoint.__init__(self, session, ep_name, address)
        HasDraftsEndpoint.__init__(self, self.url, self.session, address, ["name", "description"])
        HasSubEndpointProvider.__init__(self, self.url)
    def get_all(self):
        '''gets all queues'''
        return get(self.session, self.url)
    def create(self, group, name, description):
        '''create a queue'''
        d = {"group": group,
             "name": name,
             "description": description
            }
        return post(self.session, self.url, d)
    def modify_by_id(self, queue_id, name, description):
        '''modify a queue by id'''
        d = {"name": name,
             "description": description
            }
        return put(self.session, self.url, d, str(queue_id))
    def delete_by_id(self, queue_id):
        '''delete a queue by id'''
        d = None
        return delete(self.session, self.url, d, str(queue_id))
    def get_by_id(self, queue_id):
        '''get a queue by id'''
        return get(self.session, self.url, str(queue_id))
    
class TagsClient(Endpoint):
    def get_all(self):
        return get(self.session, self.url)
    def create(self,group, name):
        d = {"name": name, 
             "group": group}
        return post(self.session, self.url, d)
    def delete_by_id(self, tag_id):
        d = None
        return delete(self.session, self.url, d, str(tag_id))
    def get_by_id(self, tag_id):
        return get(self.session, self.url, str(tag_id))
    def modify_by_id(self, tag_id, name):
        d = {"name": name}
        return put(self.session, self.url, d, str(tag_id))
    def get_resources_by_id(self, tag_id):
        return get(self.session, self.url, str(tag_id), 'resources')

class EntrypointsClient(Endpoint, HasTagsProvider, HasDraftsEndpoint, HasSubEndpointProvider):
    def __init__(self, session, ep_name, address):
        Endpoint.__init__(self, session, ep_name, address)
        HasTagsProvider.__init__(self, self.url, self.session)
        HasDraftsEndpoint.__init__(self, self.url, self.session, address,
                                      ["name", "description", "taskGraph", "parameters", "queues", "plugins"]
                                  )
        HasSubEndpointProvider.__init__(self, self.url)
    def get_all(self):
        return get(self.session, self.url)
    def create(self, group, name, description, taskGraph, parameters, queues, plugins):
        d = {
            "group": group,
            "name": name,
            "description": description,
            "taskGraph": taskGraph,
            "parameters": parameters,
            "queues": queues,
            "plugins": plugins
        }
        return post(self.session, self.url, d)
    def modify_by_id(self, entrypoint_id, name, description, taskGraph, parameters, queues):
        d = {
            "name": name,
            "description": description,
            "taskGraph": taskGraph,
            "parameters": parameters,
            "queues": queues
        } 
        return put(self.session, self.url, d, str(entrypoint_id))
    def get_by_id(self, entrypoint_id):
        return get(self.session, self.url, str(entrypoint_id))
    def delete_by_id(self, entrypoint_id):
        d = None
        return delete(self.session, self.url, d, str(entrypoint_id))
    def get_plugins_by_entrypoint_id(self, entrypoint_id):
        return get(self.session, self.url, str(entrypoint_id), 'plugins')
    def add_plugins_by_entrypoint_id(self, entrypoint_id, plugins):
        d = {"plugins": plugins}
        return post(self.session, self.url, d, str(entrypoint_id), 'plugins')
    def get_plugins_by_entrypoint_id_plugin_id(self, entrypoint_id, plugin_id):
        return get(self.session, self.url, str(entrypoint_id), 'plugins', str(plugin_id))
    def delete_plugins_by_entrypoint_id_plugin_id(self, entrypoint_id, plugin_id):
        d = None
        return delete(self.session, self.url, d, str(entrypoint_id), 'plugins', str(plugin_id))
    def modify_queues_by_entrypoint_id(self, entrypoint_id, ids):
        d = { "ids": ids}
        return put(self.session, self.url, d, str(entrypoint_id), 'queues')
    def add_queues_by_entrypoint_id(self, entrypoint_id, ids):
        d = { "ids": ids}
        return post(self.session, self.url, d, str(entrypoint_id), 'queues')
    def get_queues_by_entrypoint_id(self, entrypoint_id):
        return get(self.session, self.url, str(entrypoint_id), 'queues')
    def delete_queues_by_entrypoint_id(self, entrypoint_id):
        d = None
        return delete(self.session, self.url, d, str(entrypoint_id), 'queues')
    def delete_queues_by_entrypoint_id_queue_id(self, entrypoint_id, queue_id):
        d = None
        return delete(self.session, self.url, d, str(entrypoint_id), 'queues', str(queue_id))
    def get_snapshots_by_entrypoint_id(self, entrypoint_id):
        return get(self.session, self.url, str(entrypoint_id), 'snapshots')
    def get_snapshots_by_entrypoint_id_snapshot_id(self, entrypoint_id, snapshot_id):
        return get(self.session, self.url, str(entrypoint_id), 'snapshots', str(snapshot_id))

class ExperimentsClient(Endpoint, HasTagsProvider, HasDraftsEndpoint, HasSubEndpointProvider):
    def __init__(self, session, ep_name, address):
        Endpoint.__init__(self, session, ep_name, address)
        HasTagsProvider.__init__(self, self.url, self.session)
        HasDraftsEndpoint.__init__(self, self.url, self.session, address,
                                      ["name", "description", "entrypoints"]
                                  )
        HasSubEndpointProvider.__init__(self, self.url)
    def get_all(self):
        return get(self.session, self.url)
    def create(self, group, name, description, entrypoints):
        d = {
            "group": group,
            "name": name,
            "description": description,
            "entrypoints": entrypoints
        }
        return post(self.session, self.url, d)
    def get_drafts(self):
        return get(self.session, self.url, 'drafts')
    def get_by_id(self, experiment_id):
        return get(self.session, self.url, str(experiment_id))
    def modify_by_id(self, experiment_id, name, description, entrypoints):
        d = {
            "name": name,
            "description": description,
            "entrypoints": entrypoints
        }
        return put(self.session, self.url, d, str(experiment_id))
    def delete_by_id(self, experiment_id):
        d = None
        return delete(self.session, self.url, d, str(experiment_id))
    def get_entrypoints_by_experiment_id(self, experiment_id):
        return get(self.session, self.url, str(experiment_id), 'entrypoints')
    def modify_entrypoints_by_experiment_id(self, experiment_id, ids):
        d = { "ids":ids }
        return put(self.session, self.url, d, str(experiment_id), 'entrypoints')
    def add_entrypoints_by_experiment_id(self, experiment_id, ids):
        d = { "ids":ids }
        return post(self.session, self.url, d, str(experiment_id), 'entrypoints')
    def delete_entrypoints_by_experiment_id(self, experiment_id):
        d = None
        return delete(self.session, self.url, d, str(experiment_id), 'entrypoints')
    def delete_entrypoints_by_experiment_id_entrypoint_id(self, experiment_id, entrypoint_id):
        d = None
        return delete(self.session, self.url, d, str(experiment_id), 'entrypoints', str(entrypoint_id))
    def get_jobs_by_experiment_id(self, experiment_id):
        return get(self.session, self.url, str(experiment_id), 'jobs')
    def create_jobs_by_experiment_id(self, experiment_id, description, queue, entrypoint, values, timeout):
        d = { 
            "description": description,
            "queue": queue,
            "entrypoint": entrypoint,
            "values": values,
            "timeout": timeout
        }
        return post(self.session, self.url, d, str(experiment_id), 'jobs')
    def get_jobs_by_experiment_id_job_id(self, experiment_id, job_id):
        return get(self.session, self.url, str(experiment_id), 'jobs', str(job_id))
    def delete_jobs_by_experiment_id_job_id(self, experiment_id, job_id):
        d = None
        return delete(self.session, self.url, d, str(experiment_id), 'jobs', str(job_id))
    def get_jobs_status_by_experiment_id_job_id(self, experiment_id, job_id):
        return get(self.session, self.url, str(experiment_id), 'jobs', str(job_id), 'status')
    def modify_jobs_status_by_experiment_id_job_id(self, experiment_id, job_id, status):
        d = {"status":status}
        return put(self.session, self.url, d, str(experiment_id), 'jobs', str(job_id), 'status')
    def get_snapshots_by_experiment_id(self, experiment_id):
        return get(self.session, self.url, str(experiment_id), 'snapshots')
    def get_snapshots_by_experiment_id_snapshot_id(self, experiment_id, snapshot_id):
        return get(self.session, self.url, str(experiment_id), 'snapshots', str(snapshot_id))

class JobsClient(Endpoint, HasTagsProvider):
    def __init__(self, session, ep_name, address):
        Endpoint.__init__(self, session, ep_name, address)
        HasTagsProvider.__init__(self, self.url, self.session)
    def get_all(self):
        return get(self.session, self.url)
    def delete_by_id(self, job_id):
        d = None
        return delete(self.session, self.url, d, str(job_id))
    def get_by_id(self, job_id):
        return get(self.session, self.url, str(job_id))
    def get_snapshots_by_job_id(self, job_id):
        return get(self.session, self.url, str(job_id), 'snapshots')
    def get_snapshots_by_job_id_snapshot_id(self, job_id, snapshot_id):
        return get(self.session, self.url, str(job_id), 'snapshots', str(snapshot_id))
    def get_status_by_job_id(self, job_id):
        return get(self.session, self.url, str(job_id), 'status')


class PluginsClient(Endpoint, HasDraftsEndpoint, HasSubEndpointProvider, HasTagsProvider):
    def __init__(self, session, ep_name, address):
        Endpoint.__init__(self, session, ep_name, address)
        HasTagsProvider.__init__(self, self.url, self.session)
        HasDraftsEndpoint.__init__(self, self.url, self.session, address,
                                      ["name", "description"]
                                  )
        HasSubEndpointProvider.__init__(self, self.url)
        self._files = PluginFilesClient(self, session, "files", address)
    @property
    def files(self):
        return self._files
    def get_all(self):
        return get(self.session, self.url)
    def create(self, group, name, description):
        d = {"group":group, 
             "name":name, 
             "description":description}
        return post(self.session, self.url, d)
    def get_by_id(self, plugin_id):
        return get(self.session, self.url, str(plugin_id))
    def modify_by_id(self, plugin_id, name, description):
        d = {"name": name,
             "description": description}
        return put(self.session, self.url, d, str(plugin_id))
    def delete_by_id(self, plugin_id):
        d = None
        return delete(self.session, self.url, d, str(plugin_id))

    def get_snapshots_by_plugin_id(self, plugin_id):
        return get(self.session, self.url, str(plugin_id), 'snapshots')
    def get_snapshot_by_plugin_id_snapshot_id(self, plugin_id, snapshot_id):
        return get(self.session, self.url, str(plugin_id), 'snapshots', str(snapshot_id))

class PluginFilesClient(SubEndpoint):
    def __init__(self, parent, session, ep_name, address):
        SubEndpoint.__init__(self, parent, session, ep_name, address)
        #HasTagsProvider.__init__(self, self.url, self.session)
        #HasDraftsEndpoint.__init__(self, self.url, self.session, address,
        #                              ["filename", "description"]
        #                          )
        #HasSubEndpointProvider.__init__(self, self.url)
    def get_files_by_plugin_id(self, plugin_id):
        return get(self.session, self.suburl(plugin_id))
    def create_files_by_plugin_id(self, plugin_id, filename, contents, description, *plugins):
        d = {"filename": filename,
             "contents": contents,
             "description": description,
             "tasks": [plugin.as_dict() for plugin in plugins]
            }
        return post(self.session, self.suburl(plugin_id), d)
    def delete_files_by_plugin_id(self, plugin_id):
        d = None
        return delete(self.session, self.suburl(plugin_id), d)
    def get_files_drafts_by_plugin_id(self, plugin_id):
        return get(self.session, self.suburl(plugin_id), 'drafts')
    def create_files_drafts_by_plugin_id(self, plugin_id, filename, contents, description, *plugins):
        d = {"filename": filename,
             "contents": contents,
             "description": description,
             "tasks": [plugin.as_dict() for plugin in plugins]
            }
        return post(self.session, self.suburl(plugin_id), d, 'drafts')
    def get_files_drafts_by_plugin_id_draft_id(self, plugin_id, drafts_id):
        return get(self.session, self.suburl(plugin_id), 'drafts', str(drafts_id))
    def modify_files_drafts_by_plugin_id_draft_id(self, plugin_id, drafts_id, filename, contents, description, *plugins):
        d = {"filename": filename,
             "contents": contents,
             "description": description,
             "tasks": [plugin.as_dict() for plugin in plugins]
            }
        return put(self.session, self.suburl(plugin_id), d, 'drafts', str(drafts_id))
    def delete_files_drafts_by_plugin_id_draft_id(self, plugin_id, drafts_id):
        d = None
        return delete(self.session, self.suburl(plugin_id), d, 'drafts', str(drafts_id))
    def get_files_by_plugin_id_file_id(self, plugin_id, file_id):
        return get(self.session, self.suburl(plugin_id), str(file_id))
    def modify_files_by_plugin_id_file_id(self, plugin_id, file_id, filename, contents, description, *plugins):
        d = {"filename": filename,
             "contents": contents,
             "description": description,
             "tasks": [plugin.as_dict() for plugin in plugins]
            }
        return put(self.session, self.suburl(plugin_id), d, str(file_id))
    def delete_files_by_plugin_id_file_id(self, plugin_id, file_id):
        d = None
        return delete(self.session, self.suburl(plugin_id), d, str(file_id))
    def get_files_draft_by_plugin_id_file_id(self, plugin_id, file_id):
        return get(self.session, self.suburl(plugin_id), str(file_id), 'draft')
    def modify_files_draft_by_plugin_id_file_id(self, plugin_id, file_id, filename, contents, description, *plugins):
        d = {"filename": filename,
             "contents": contents,
             "description": description,
             "tasks": [plugin.as_dict() for plugin in plugins]
            }
        return put(self.session, self.suburl(plugin_id), d, str(file_id), 'draft')
    def delete_files_draft_by_plugin_id_file_id(self, plugin_id, file_id):
        d = None
        return delete(self.session, self.suburl(plugin_id), d, str(file_id), 'draft')
    def create_files_draft_by_plugin_id_file_id(self, plugin_id, file_id, filename, contents, description, *plugins):
        d = {"filename": filename,
             "contents": contents,
             "description": description,
             "tasks": [plugin.as_dict() for plugin in plugins]
            }
        return post(self.session, self.suburl(plugin_id), d, str(file_id), 'draft')
    def get_snapshots_by_plugin_id_file_id(self, plugin_id, file_id):
        return get(self.session, self.suburl(plugin_id), str(file_id), 'snapshots')
    def get_snapshots_by_plugin_id_file_id_snapshot_id(self, plugin_id, file_id, snapshot_id):
        return get(self.session, self.suburl(plugin_id), str(file_id), 'snapshots', str(snapshot_id))
    def get_tags_by_plugin_id_file_id(self, plugin_id, file_id):
        return get(self.session, self.suburl(plugin_id), str(file_id), 'tags')
    def modify_tags_by_plugin_id_file_id(self, plugin_id, file_id, ids):
        d = {"ids":ids}
        return put(self.session, self.suburl(plugin_id), d, str(file_id), 'tags')
    def delete_tags_by_plugin_id_file_id(self, plugin_id, file_id):
        d = None
        return delete(self.session, self.suburl(plugin_id), d, str(file_id), 'tags')
    def add_tags_by_plugin_id_file_id(self, plugin_id, file_id, ids):
        d = {"ids":ids}
        return post(self.session, self.suburl(plugin_id), d, str(file_id), 'tags')
    def delete_tags_by_plugin_id_file_id_tag_id(self, plugin_id, file_id, tag_id):
        d = None
        return delete(self.session, self.suburl(plugin_id), d, str(file_id), 'tags', str(tag_id))
        
class PluginParameterTypesClient(Endpoint):
    def get_all(self):
        return get(self.session, self.url)
    def create(self, group, name, description, structure):
        d = {"group": group,
             "name": name,
             "description": description,
             "structure": structure}
        return post(self.session, self.url, d)
    def get_by_id(self, type_id):
        return get(self.session, self.url, str(type_id))
    def modify_by_id(self, type_id, name, description, structure):
        d = {"name": name,
             "description": description,
             "structure": structure}
        return put(self.session, self.url, d, str(type_id))
    def delete_by_id(self, type_id):
        d = None
        return delete(self.session, self.url, d, str(type_id))

class PluginTask(object):
    def __init__(self, name, inputs, outputs, client):
        self.name = name
        self.inputs = inputs # expects [(name1, type1), (name2, type2) ...]
        self.outputs = outputs # expects [(name1, type1), (name2, type2) ...]
        self.client = client
    def convert_params_to_ids(self, mappings):
        '''this converts parameters to registered ids using a mapping from register_unregistered_types'''
        return [(i[0], mappings[i[1]]) for i in self.inputs], [(o[0], mappings[o[1]]) for o in self.outputs]
    def register_unregistered_types(self, group=1):
        '''checks all the types in inputs/outputs and register things that aren't registered'''
        registered_types = self.client.pluginParameterTypes.get_all() # get all registered types
        types_used_in_plugin = set([m[1] for m in self.inputs] + [m[1] for m in self.outputs]) # get all types for this plugin
        types_to_id = {}
        for registered in registered_types['data']: # add registered types to our dictionary
            types_to_id[str(registered['name'])] = str(registered['id'])
        for used in types_used_in_plugin:
            used = str(used)
            if (used not in types_to_id): # not yet registered, so register it
                response = self.client.pluginParameterTypes.create(group, used, used + " plugin parameter", structure={})
                types_to_id[used] = str(response['id'])
        return types_to_id # mapping of types to ids
    def as_dict(self, mappings=None):
        '''convert it to a dict to be sent to the RESTAPI'''
        if mappings is None:
            mappings = self.register_unregistered_types()
        ins, outs = self.convert_params_to_ids(mappings)
        return {
            "name": self.name,
            "inputParams": [{"name": param[0], "parameterType": param[1]} for param in ins],
            "outputParams": [{"name": param[0], "parameterType": param[1]} for param in outs]
        }

class ArtifactsClient(Endpoint):
    def get_all(self):
        return get(self.session, self.url)
    def create(self, group, description, job, uri):
        d = {"group": group,
             "description": description,
             "job": job,
             "uri": uri}
        return post(self.session, self.url, d)
    def get_by_id(self, artifact_id):
        return get(self.session, self.url, str(artifact_id))
    def modify_by_id(self, artifact_id, description):
        d = {"description": description}
        return put(self.session, self.url, d, str(artifact_id))
    def get_snapshots(self, artifact_id):
        return get(self.session, self.url, str(artifact_id), 'snapshots')
    def get_snapshots_by_artifact_id_snapshot_id(self, artifact_id, snapshot_id):
        return get(self.session, self.url, str(artifact_id), 'snapshots', str(snapshot_id))
class ModelsClient(Endpoint, HasTagsProvider, HasDraftsEndpoint,HasSubEndpointProvider):
    def __init__(self, session, ep_name, address):
        Endpoint.__init__(self, session, ep_name, address)
        HasSubEndpointProvider.__init__(self, self.url)
        HasTagsProvider.__init__(self, self.url, self.session)
        HasDraftsEndpoint.__init__(self, self.url, self.session, address,
                                      ["name", "description"]
                                  )
    def get_all(self):
        return get(self.session, self.url)
    def create(self, group, name, description):
        d = {"group": group,
             "name": name,
             "description": description}
        return post(self.session, self.url, d)
    def get_by_id(self, model_id):
        return get(self.session, self.url, str(model_id))
    def modify_by_id(self, model_id, name, description):
        d = {"name": name,
             "description": description}
        return put(self.session, self.url, d, str(model_id))
    def delete_by_id(self, model_id):
        d = None
        return delete(self.session, self.url, d, str(model_id))
    def get_snapshots_by_model_id(self, model_id):
        return get(self.session, self.url, str(model_id), 'snapshots')
    def get_snapshot_by_plugin_id_model_id(self, model_id, snapshot_id):
        return get(self.session, self.url, str(model_id), 'snapshots', str(snapshot_id))
    def get_versions_by_model_id(self, model_id):
        return get(self.session, self.url, str(model_id), 'versions')
    def create_version_by_model_id(self, model_id, description, artifact):
        d = {"description": description,
             "artifact": artifact}
        return post(self.session, self.url, d, str(model_id), 'versions')
    def modify_version_by_model_id_version_id(self, model_id, version_id, description):
        d = {"description": description}
        return put(self.session, self.url, d, str(model_id), 'versions', str(version_id))
    def get_version_by_model_id_version_id(self, model_id, version_id):
        return get(self.session, self.url, str(model_id), 'versions', str(version_id))

  

class DraftsEndpoint(SubEndpoint):
    def __init__(self, base_url, parent, session, ep_name, address):
        SubEndpoint.__init__(self, parent, session, ep_name, address)
        self.base_url = base_url
        self.fields = parent.draft_fields # array of field names
        self.put_fields = parent.put_fields # used when PUT method differs from create
    @property
    def drafts_url(self):
        return urljoin(self.base_url, 'drafts')
    
    ## /something/id/draft
    
    def create_draft_for_resource(self, parent_id, *fields): #TODO: what to do about these parameters? they can be different
        d = {}
        for f in zip(self.fields, fields):
            d[f[0]] = f[1]
        return post(self.session, self.suburl(parent_id), d)
    def get_draft_for_resource(self, parent_id):
        return get(self.session, self.suburl(parent_id))
    def modify_draft_for_resource(self, parent_id, *fields):
        d = {}
        for f in zip(self.put_fields, fields):
            d[f[0]] = f[1]
        return put(self.session, self.suburl(parent_id), d)
    def delete_draft_for_resource(self, parent_id):
        d = None
        return delete(self.session, self.suburl(parent_id), d)

    ## /something/drafts/
    
    def get_all(self):
        return get(self.session, self.drafts_url)
    def create(self, group_id, *fields):
        d = {"group": group_id}
        for f in zip(self.fields, fields):
            d[f[0]] = f[1]
        return post(self.session, self.drafts_url, d)
    def modify_by_draft_id(self, draft_id, *fields):
        d = {}
        for f in zip(self.put_fields, fields):
            d[f[0]] = f[1]
        return put(self.session, self.drafts_url, d, str(draft_id))
    def delete_by_draft_id(self, draft_id):
        d = None
        return delete(self.session, self.drafts_url, d, str(draft_id))
    def get_by_draft_id(self, draft_id):
        return get(self.session, self.drafts_url, str(draft_id))

class TagsProvider(object):
    def __init__(self, base_url, session):
        #SubEndpoint.__init__(self, session)
        self.url = base_url 
        self.session = session
    def get(self, parent_id):
        return get(self.session, self.url, str(parent_id), 'tags')
    def modify(self, parent_id, ids):
        d = {"ids":ids}
        return put(self.session, self.url, d, str(parent_id), 'tags')
    def delete_all(self, parent_id):
        d = None
        return delete(self.session, self.url, d, str(parent_id), 'tags')
    def add(self, parent_id, ids):
        d = {"ids":ids}
        return post(self.session, self.url, d, str(parent_id), 'tags')
    def delete(self, parent_id, tag_id):
        d = None
        return delete(self.session, self.url, d, str(parent_id), 'tags', str(tag_id))


In [3]:
client = DioptraClient(requests.Session())

In [4]:
def this_is_my_plugin_task(x, y):
    return y, x

try:
    client.users.create('pluginuser','pluginuser@dioptra.nccoe.nist.gov','pleasemakesuretoPLUGINthecomputer','pleasemakesuretoPLUGINthecomputer')
except:
    pass # ignore if user exists already
client.auth.login('pluginuser','pleasemakesuretoPLUGINthecomputer')

# first let's create a PluginTask for our plugin
my_plugin = PluginTask(name='this_is_my_plugin_task', inputs=[('x', 'int'), ('y', 'str')], outputs=[('y', 'str'), ('x', 'int')], client=client)
#my_plugin = PluginTask(name='this_is_my_plugin_task', inputs=[('x', 'int')], outputs=[('y', 'str')], client=client)
plugin_id = client.plugins.create(1,'first_plugin', 'this plugin does nothing')['id']

try: 
    tag_id = client.tags.create('youreit', 1)['id']
except:
    pass
try:
    tag_id2 = client.tags.create('nuhuh', 1)['id']
except:
    pass

#old
#client.plugins.add_tags_by_plugin_id(plugin_id, [tag_id])
#client.plugins.modify_tags_by_plugin_id(plugin_id, [tag_id, tag_id2])
#client.plugins.get_tags_by_plugin_id(plugin_id)
#client.plugins.delete_tags_by_plugin_id_tag_id(plugin_id, tag_id)
#client.plugins.delete_tags_by_plugin_id(plugin_id)

####

#client.plugins.tags.add(plugin_id, [tag_id])
#client.plugins.tags.modify(plugin_id, [tag_id, tag_id2])
#client.plugins.tags.get(plugin_id)
#client.plugins.tags.delete(plugin_id, tag_id)
#client.plugins.tags.delete_all(plugin_id)
#client.plugins.delete_by_id(plugin_id)




2024-07-22 11:08:21 [debug    ] Request made.                  data={'username': 'pluginuser', 'email': 'pluginuser@dioptra.nccoe.nist.gov', 'password': 'pleasemakesuretoPLUGINthecomputer', 'confirmPassword': 'pleasemakesuretoPLUGINthecomputer'} method=POST url=http://localhost:20080/api/v1/users/
2024-07-22 11:08:21 [error    ] Error code 400 returned.       data={'username': 'pluginuser', 'email': 'pluginuser@dioptra.nccoe.nist.gov', 'password': 'pleasemakesuretoPLUGINthecomputer', 'confirmPassword': 'pleasemakesuretoPLUGINthecomputer'} method=POST response={"message": "Bad Request - The username on the registration form is not available. Please select another and resubmit."}
 url=http://localhost:20080/api/v1/users/
2024-07-22 11:08:21 [debug    ] Request made.                  data={'username': 'pluginuser', 'password': 'pleasemakesuretoPLUGINthecomputer'} method=POST url=http://localhost:20080/api/v1/auth/login
2024-07-22 11:08:21 [debug    ] Response received.             json={'

In [5]:
def delete_all():
    for d in client.experiments.get_all()['data']:
        client.experiments.delete_by_id(d['id'])
    for d in client.entrypoints.get_all()['data']:
         client.entrypoints.delete_by_id(d['id'])
    for d in client.jobs.get_all()['data']:
         client.jobs.delete_by_id(d['id'])
    for d in client.models.get_all()['data']:
         client.models.delete_by_id(d['id'])
    for d in client.plugins.get_all()['data']:
        try:
            client.plugins.delete_by_id(d['id'])
        except:
            pass
    for d in client.tags.get_all()['data']:
         client.tags.delete_by_id(d['id'])
    for d in client.pluginParameterTypes.get_all()['data']:
        try:
            client.pluginParameterTypes.delete_by_id(d['id'])
        except:
            pass
    for d in client.queues.get_all()['data']:
        client.queues.delete_by_id(d['id'])


In [6]:
delete_all()

2024-07-22 11:08:21 [debug    ] Request made.                  data=None method=GET url=http://localhost:20080/api/v1/experiments/
2024-07-22 11:08:21 [debug    ] Response received.             json={'index': 0, 'isComplete': True, 'totalNumResults': 1, 'first': '/api/v1/experiments?index=0&pageLength=10', 'data': [{'id': 149, 'snapshot': 216, 'group': {'id': 1, 'name': 'public', 'url': '/api/v1/groups/1'}, 'user': {'id': 5, 'username': 'finalman', 'url': '/api/v1/users/5'}, 'createdOn': '2024-07-22T15:05:44.022894+00:00', 'lastModifiedOn': '2024-07-22T15:05:44.022894+00:00', 'latestSnapshot': True, 'hasDraft': False, 'tags': [], 'name': 'dioptra v1s first experiment', 'description': 'modded', 'entrypoints': [{'id': 157, 'group': {'id': 1, 'name': 'public', 'url': '/api/v1/groups/1'}, 'url': '/api/v1/entrypoints/157', 'name': 'new name'}]}]}
2024-07-22 11:08:21 [debug    ] Request made.                  data=None method=DELETE url=http://localhost:20080/api/v1/experiments/149
2024-07-2

In [7]:
try:
    # these might fail if this has been run before
    client.users.create('testuser','testuser@gmail.com','testuserpassword','testuserpassword')
except:
    pass
try:
    client.users.create('testuser2','testuser2@gmail.com','testuserpassword','testuserpassword')
except:
    pass
client.auth.login('testuser','testuserpassword')
client.users.get_all()
client.users.current()
client.users.update_password_by_id(client.users.current()['id'],'testuserpassword','newtestuserpassword','newtestuserpassword')
try:
    client.users.modify_current('testuser','newemail@email.com') # this one should fail because we are not logged in after a password change
except:
    pass
client.auth.login('testuser','newtestuserpassword')
client.users.modify_current('testuser','newemail@email.com')
client.users.modify_current_password('newtestuserpassword','newnewtestuserpassword','newnewtestuserpassword')
try: 
    client.auth.logout(True)
except:
    pass
client.auth.login('testuser','newnewtestuserpassword')
client.users.delete_current('newnewtestuserpassword')
try: 
    client.users.failed_user_post() # if we get the schema wrong, it does return JSON, raise if not 200, but send json back to logs
except:
    pass
try:
    client.users.failed_user_get() # if we get the URL wrong, it does not return JSON
except:
    pass
client.auth.login('testuser2','testuserpassword')
client.users.get_by_id(client.users.current()['id'])


2024-07-22 11:08:22 [debug    ] Request made.                  data={'username': 'testuser', 'email': 'testuser@gmail.com', 'password': 'testuserpassword', 'confirmPassword': 'testuserpassword'} method=POST url=http://localhost:20080/api/v1/users/
2024-07-22 11:08:22 [debug    ] Response received.             json={'username': 'testuser', 'email': 'testuser@gmail.com', 'id': 8, 'groups': [{'id': 1, 'name': 'public', 'url': '/api/v1/groups/1'}], 'createdOn': '2024-07-22T15:08:22.141744+00:00', 'lastModifiedOn': '2024-07-22T15:08:22.141744+00:00', 'lastLoginOn': None, 'passwordExpiresOn': '2025-07-22T15:08:22.141744+00:00'}
2024-07-22 11:08:22 [debug    ] Request made.                  data={'username': 'testuser2', 'email': 'testuser2@gmail.com', 'password': 'testuserpassword', 'confirmPassword': 'testuserpassword'} method=POST url=http://localhost:20080/api/v1/users/
2024-07-22 11:08:22 [error    ] Error code 400 returned.       data={'username': 'testuser2', 'email': 'testuser2@gmail.

{'username': 'testuser2', 'email': 'testuser2@gmail.com', 'id': 3}

In [8]:
try:
    client.users.create('queuemgr','queuemgr@dioptra.nccoe.nist.gov','lmnopQUEUErs','lmnopQUEUErs')
except:
    pass # ignore if user exists already
client.auth.login('queuemgr','lmnopQUEUErs')
cpu_id = client.queues.create(1, 'tensorflow_cpu2', 'for running tensorflow on a cpu')['id']
client.queues.modify_by_id(cpu_id, 'tensorflow_cpu2', 'for running tensorflow on a cpu - modified')
client.queues.get_all()
gpu_draft_id = client.queues.drafts.create(1, 'tensorflow_gpu', 'for running tensorflow on a gpu')['id']
client.queues.drafts.modify_by_draft_id(gpu_draft_id, 'tensorflow_gpu', 'for running tensorflow on a gpu - modified')
client.queues.drafts.get_all()
client.queues.drafts.get_by_draft_id(gpu_draft_id)
client.queues.drafts.create_draft_for_resource(cpu_id, 'tensorflow_cpu2', 'oops my draft deleted the description')
client.queues.drafts.get_draft_for_resource(cpu_id)
client.queues.drafts.modify_draft_for_resource(cpu_id, 'tensorflow_cpu2', 'wait go back')
client.queues.drafts.delete_draft_for_resource(cpu_id)
client.queues.drafts.delete_by_draft_id(gpu_draft_id)


2024-07-22 11:08:22 [debug    ] Request made.                  data={'username': 'queuemgr', 'email': 'queuemgr@dioptra.nccoe.nist.gov', 'password': 'lmnopQUEUErs', 'confirmPassword': 'lmnopQUEUErs'} method=POST url=http://localhost:20080/api/v1/users/
2024-07-22 11:08:22 [error    ] Error code 400 returned.       data={'username': 'queuemgr', 'email': 'queuemgr@dioptra.nccoe.nist.gov', 'password': 'lmnopQUEUErs', 'confirmPassword': 'lmnopQUEUErs'} method=POST response={"message": "Bad Request - The username on the registration form is not available. Please select another and resubmit."}
 url=http://localhost:20080/api/v1/users/
2024-07-22 11:08:22 [debug    ] Request made.                  data={'username': 'queuemgr', 'password': 'lmnopQUEUErs'} method=POST url=http://localhost:20080/api/v1/auth/login
2024-07-22 11:08:22 [debug    ] Response received.             json={'username': 'queuemgr', 'status': 'Login successful'}
2024-07-22 11:08:22 [debug    ] Request made.                 

{'id': [67], 'status': 'Success'}

In [9]:
client.tags.get_all()
tagid = client.tags.create(1, 'tag_name')['id']
client.tags.get_all()
client.tags.get_by_id(tagid)
client.tags.modify_by_id(tagid, 'new_name')
client.tags.get_resources_by_id(tagid)
client.tags.delete_by_id(tagid)
client.tags.get_all()

2024-07-22 11:08:22 [debug    ] Request made.                  data=None method=GET url=http://localhost:20080/api/v1/tags/
2024-07-22 11:08:22 [debug    ] Response received.             json={'index': 0, 'isComplete': True, 'totalNumResults': 0, 'first': '/api/v1/tags?index=0&pageLength=10', 'data': []}
2024-07-22 11:08:22 [debug    ] Request made.                  data={'name': 'tag_name', 'group': 1} method=POST url=http://localhost:20080/api/v1/tags/
2024-07-22 11:08:22 [debug    ] Response received.             json={'id': 33, 'group': {'id': 1, 'name': 'public', 'url': '/api/v1/groups/1'}, 'user': {'id': 4, 'username': 'queuemgr', 'url': '/api/v1/users/4'}, 'createdOn': '2024-07-22T15:08:22.682278+00:00', 'lastModifiedOn': '2024-07-22T15:08:22.682278+00:00', 'name': 'tag_name'}
2024-07-22 11:08:22 [debug    ] Request made.                  data=None method=GET url=http://localhost:20080/api/v1/tags/
2024-07-22 11:08:22 [debug    ] Response received.             json={'index': 0, 

{'index': 0,
 'isComplete': True,
 'totalNumResults': 0,
 'first': '/api/v1/tags?index=0&pageLength=10',
 'data': []}

In [10]:
delete_all()

2024-07-22 11:08:23 [debug    ] Request made.                  data=None method=GET url=http://localhost:20080/api/v1/experiments/
2024-07-22 11:08:23 [debug    ] Response received.             json={'index': 0, 'isComplete': True, 'totalNumResults': 0, 'first': '/api/v1/experiments?index=0&pageLength=10', 'data': []}
2024-07-22 11:08:23 [debug    ] Request made.                  data=None method=GET url=http://localhost:20080/api/v1/entrypoints/
2024-07-22 11:08:23 [debug    ] Response received.             json={'index': 0, 'isComplete': True, 'totalNumResults': 0, 'first': '/api/v1/entrypoints?index=0&pageLength=10', 'data': []}
2024-07-22 11:08:23 [debug    ] Request made.                  data=None method=GET url=http://localhost:20080/api/v1/jobs/
2024-07-22 11:08:23 [debug    ] Response received.             json={'index': 0, 'isComplete': True, 'totalNumResults': 0, 'first': '/api/v1/jobs?index=0&pageLength=10', 'data': []}
2024-07-22 11:08:23 [debug    ] Request made.         

In [11]:
def this_is_my_plugin_task(x, y):
    return y, x

try:
    client.users.create('pluginuser','pluginuser@dioptra.nccoe.nist.gov','pleasemakesuretoPLUGINthecomputer','pleasemakesuretoPLUGINthecomputer')
except:
    pass # ignore if user exists already
client.auth.login('pluginuser','pleasemakesuretoPLUGINthecomputer')

# first let's create a PluginTask for our plugin
my_plugin = PluginTask(name='this_is_my_plugin_task', inputs=[('x', 'int'), ('y', 'str')], outputs=[('y', 'str'), ('x', 'int')], client=client)
#my_plugin = PluginTask(name='this_is_my_plugin_task', inputs=[('x', 'int')], outputs=[('y', 'str')], client=client)
plugin_id = client.plugins.create(1,'first_plugin', 'this plugin does nothing')['id']
client.plugins.get_all()
client.plugins.get_by_id(plugin_id)
client.plugins.modify_by_id(plugin_id, 'first_plugin', 'this plugin still does nothing')

#### NEW

draft_id = client.plugins.drafts.create(1, 'draft_second_plugin', 'this plugin might do something someday')['id']
client.plugins.drafts.get_all()
client.plugins.drafts.get_by_draft_id(draft_id)
client.plugins.drafts.modify_by_draft_id(draft_id, 'draft_second_plugin', 'nevermind its hopeless')
client.plugins.drafts.delete_by_draft_id(draft_id)
client.plugins.drafts.create_draft_for_resource(plugin_id, 'first_plugin', 'this plugin could do something now!!')
client.plugins.drafts.modify_draft_for_resource(plugin_id, 'first_plugin', 'this plugin could actually do something now!!')
client.plugins.drafts.get_draft_for_resource(plugin_id)
client.plugins.drafts.delete_draft_for_resource(plugin_id)

#### NEW

client.plugins.files.create_files_by_plugin_id(plugin_id, 'the_current_file.py', "def this_is_my_plugin_task(x, y):\n    return y x", "i hope i didn't forget a comma", my_plugin)
client.plugins.files.get_files_by_plugin_id(plugin_id)
client.plugins.files.delete_files_by_plugin_id(plugin_id)

client.plugins.files.create_files_drafts_by_plugin_id(plugin_id, 'the_current_draft_file.py', "def this_is_my_plugin_task(x, y):\n    return y x", "i hope i didn't forget a comma", my_plugin)
fdrafts_id = client.plugins.files.get_files_drafts_by_plugin_id(plugin_id)['data'][0]['id']
client.plugins.files.modify_files_drafts_by_plugin_id_draft_id(plugin_id, fdrafts_id, 'the_current_draft_file.py', "def this_is_my_updated_plugin_task(x, y):\n    return y x", "i hope i didn't forget a comma", my_plugin)
client.plugins.files.get_files_drafts_by_plugin_id_draft_id(plugin_id, fdrafts_id)
client.plugins.files.delete_files_drafts_by_plugin_id_draft_id(plugin_id, fdrafts_id)

files_id = client.plugins.files.create_files_by_plugin_id(plugin_id, 'the_new_current_file.py', "def this_is_my_plugin_task(x, y):\n    return y x", "i hope i didn't forget a comma", my_plugin)['id']
client.plugins.files.modify_files_by_plugin_id_file_id(plugin_id, files_id, 'the_new_current_file.py', "def this_is_my_plugin_task(x, y):\n    return y, x", "i forgot a comma", my_plugin)
client.plugins.files.get_files_by_plugin_id_file_id(plugin_id, files_id)
client.plugins.files.delete_files_by_plugin_id_file_id(plugin_id, files_id)


file_id = client.plugins.files.create_files_by_plugin_id(plugin_id, 'NEW_FILE.py', "def this_is_my_plugin_task(x, y):\n    return y x", "i hope i didn't forget a comma", my_plugin)['id']
client.plugins.files.create_files_draft_by_plugin_id_file_id(plugin_id, file_id, 'newfile.py', 'def invalid_python(x,y):', 'this is a test of the emergency plugin system', my_plugin)
client.plugins.files.modify_files_draft_by_plugin_id_file_id(plugin_id, file_id, 'newfile.py', 'def invalider_python(x,y):', 'test successful', my_plugin)
client.plugins.files.get_files_draft_by_plugin_id_file_id(plugin_id, file_id)
client.plugins.files.get_snapshots_by_plugin_id_file_id(plugin_id, file_id)
# cannot test below. do not know how to make a snapshot
# client.plugins.get_snapshots_by_plugin_id_file_id_snapshot_id(plugin_id, file_id, snapshot_id)
# client.plugin.get_snapshots_by_plugin_id(plugin_id)
# client.plugin.get_snapshot_by_plugin_id_snapshot_id(plugin_id, snapshot_id) # no way to create snapshots as of now?
tag_id = client.tags.create(1,'youreit')['id']
tag_id2 = client.tags.create(1,'nuhuh')['id']
client.plugins.tags.add(plugin_id, [tag_id])
client.plugins.tags.modify(plugin_id, [tag_id, tag_id2])
client.plugins.tags.get(plugin_id)
client.plugins.tags.delete(plugin_id, tag_id)
client.plugins.tags.delete_all(plugin_id)
client.plugins.delete_by_id(plugin_id)

client.plugins.files.add_tags_by_plugin_id_file_id(plugin_id, file_id, [tag_id])
client.plugins.files.modify_tags_by_plugin_id_file_id(plugin_id, file_id, [tag_id, tag_id2])
client.plugins.files.get_tags_by_plugin_id_file_id(plugin_id, file_id)
client.plugins.files.delete_tags_by_plugin_id_file_id(plugin_id, file_id)
client.plugins.files.add_tags_by_plugin_id_file_id(plugin_id, file_id, [tag_id])
client.plugins.files.delete_tags_by_plugin_id_file_id_tag_id(plugin_id, file_id, tag_id)
client.plugins.files.delete_files_draft_by_plugin_id_file_id(plugin_id, file_id)

client.pluginParameterTypes.get_all()
type_id = client.pluginParameterTypes.create(1, 'bool', 'its a yes or no question', {})['id']
client.pluginParameterTypes.modify_by_id(type_id, 'bool', 'it cant be None. probably', {})
client.pluginParameterTypes.get_by_id(type_id)
client.pluginParameterTypes.delete_by_id(type_id)



2024-07-22 11:08:24 [debug    ] Request made.                  data={'username': 'pluginuser', 'email': 'pluginuser@dioptra.nccoe.nist.gov', 'password': 'pleasemakesuretoPLUGINthecomputer', 'confirmPassword': 'pleasemakesuretoPLUGINthecomputer'} method=POST url=http://localhost:20080/api/v1/users/
2024-07-22 11:08:24 [error    ] Error code 400 returned.       data={'username': 'pluginuser', 'email': 'pluginuser@dioptra.nccoe.nist.gov', 'password': 'pleasemakesuretoPLUGINthecomputer', 'confirmPassword': 'pleasemakesuretoPLUGINthecomputer'} method=POST response={"message": "Bad Request - The username on the registration form is not available. Please select another and resubmit."}
 url=http://localhost:20080/api/v1/users/
2024-07-22 11:08:24 [debug    ] Request made.                  data={'username': 'pluginuser', 'password': 'pleasemakesuretoPLUGINthecomputer'} method=POST url=http://localhost:20080/api/v1/auth/login
2024-07-22 11:08:24 [debug    ] Response received.             json={'

{'id': [171], 'status': 'Success'}

In [12]:
delete_all()

2024-07-22 11:08:29 [debug    ] Request made.                  data=None method=GET url=http://localhost:20080/api/v1/experiments/
2024-07-22 11:08:29 [debug    ] Response received.             json={'index': 0, 'isComplete': True, 'totalNumResults': 0, 'first': '/api/v1/experiments?index=0&pageLength=10', 'data': []}
2024-07-22 11:08:29 [debug    ] Request made.                  data=None method=GET url=http://localhost:20080/api/v1/entrypoints/
2024-07-22 11:08:29 [debug    ] Response received.             json={'index': 0, 'isComplete': True, 'totalNumResults': 0, 'first': '/api/v1/entrypoints?index=0&pageLength=10', 'data': []}
2024-07-22 11:08:29 [debug    ] Request made.                  data=None method=GET url=http://localhost:20080/api/v1/jobs/
2024-07-22 11:08:29 [debug    ] Response received.             json={'index': 0, 'isComplete': True, 'totalNumResults': 0, 'first': '/api/v1/jobs?index=0&pageLength=10', 'data': []}
2024-07-22 11:08:29 [debug    ] Request made.         

In [13]:
import random

try:
    client.users.create('finalman','finalman@dioptra.nccoe.nist.gov','fighteroftheDRAFTman','fighteroftheDRAFTman')
except:
    pass # ignore if user exists already
client.auth.login('finalman','fighteroftheDRAFTman')
exp_id = client.experiments.create(1, 'dioptra v1s first experiment', 'get the camera', [])['id']
client.experiments.get_all()
client.experiments.modify_by_id(exp_id,'dioptra v1s first experiment','modded',[])
client.experiments.get_by_id(exp_id)

draft_id = client.experiments.drafts.create(1, 'dioptra v1s first DRAFT experiment', 'dont need the camera for this', [])['id']
client.experiments.drafts.modify_by_draft_id(draft_id, 'dioptra v1s first DRAFT experiment', 'removed description', [])
client.experiments.drafts.get_all()
client.experiments.drafts.get_by_draft_id(draft_id)
client.experiments.drafts.create_draft_for_resource(exp_id,  'v1draft', 'drafty in here', [])
client.experiments.drafts.get_draft_for_resource(exp_id)
client.experiments.drafts.modify_draft_for_resource(exp_id,'editing is what drafts are for', 'drafty in here', [])
client.experiments.drafts.delete_draft_for_resource(exp_id)
client.experiments.drafts.delete_by_draft_id(draft_id)

tag_id = client.tags.create(1,'youreit')['id']
tag_id2 = client.tags.create(1,'nuhuh')['id']

try: 
    queue_id = client.queues.create(1, 'tensorflow_cpu3', 'for running tensorflow on a cpu')['id']
except:
    pass

my_plugin = PluginTask(name='this_is_my_plugin', inputs=[('x', 'int'), ('y', 'str')], outputs=[('y', 'str'), ('x', 'int')], client=client)
my_plugin2 = PluginTask(name='this_is_not_my_plugin', inputs=[('x', 'int'), ('y', 'str')], outputs=[('y', 'str'), ('x', 'int')], client=client)

try: 
    plugin_id = client.plugins.create(1,'not_the_first_plugin', 'this plugin does nothing')['id']
    files_id = client.plugins.files.create_files_by_plugin_id(plugin_id, 'the_new_current_file.py', "def this_is_my_plugin_task(x, y):\n    return y x", "i hope i didn't forget a comma", my_plugin)['id']
    client.plugins.files.get_files_by_plugin_id_file_id(plugin_id, files_id)

except:
    pass
try:
    plugin_id2 = client.plugins.create(1,'not_the_first_plugin2', 'this plugin does nothing')['id']
    files_id = client.plugins.files.create_files_by_plugin_id(plugin_id2, 'the_new_current_file.py', "def this_is_my_plugin_task(x, y):\n    return y x", "i hope i didn't forget a comma", my_plugin2)['id']
    client.plugins.files.get_files_by_plugin_id_file_id(plugin_id2, files_id)
except:
    pass

# entrypoints testing#
ep_id = client.entrypoints.create(1, 'entrypoint1', 'this is where you entry', 'task graph?',
                                  [{
                                      'name': 'parameter_name',
                                      'defaultValue': 'default_value',
                                      'parameterType': 'string'
                                  }], queues = [queue_id], plugins=[plugin_id])['id']
client.entrypoints.get_all()
client.entrypoints.modify_by_id(ep_id, 'new name', 'new desc', 'task graph?',
                                    [{
                                        'name': 'parameter_name',
                                        'defaultValue': 'default_value',
                                        'parameterType': 'string'
                                    }], queues = [queue_id])  # TODO: no plugins? correct?
client.entrypoints.get_by_id(ep_id)

###
draft_id = client.entrypoints.drafts.create(1, 'entrypoint1', 'this is where you entry', 'task graph?',
                                  [{
                                      'name': 'parameter_name',
                                      'defaultValue': 'default_value',
                                      'parameterType': 'string'
                                  }], [queue_id], [plugin_id])['id']
client.entrypoints.drafts.modify_by_draft_id(draft_id, 'new name', 'new desc', 'task graph?',
                                    [{
                                        'name': 'parameter_name',
                                        'defaultValue': 'default_value',
                                        'parameterType': 'string'
                                    }], [queue_id])
client.entrypoints.drafts.get_all()
client.entrypoints.drafts.get_by_draft_id(draft_id)
client.entrypoints.drafts.create_draft_for_resource(ep_id, 'new name', 'new desc', 'task graph?',
                                    [{
                                        'name': 'parameter_name',
                                        'defaultValue': 'default_value',
                                        'parameterType': 'string'
                                    }], [queue_id], [plugin_id])
client.entrypoints.drafts.get_draft_for_resource(ep_id)
client.entrypoints.drafts.modify_draft_for_resource(ep_id, 'new new name', 'new desc', 'task graph?',
                                    [{
                                        'name': 'parameter_name',
                                        'defaultValue': 'default_value',
                                        'parameterType': 'string'
                                    }], [queue_id])
client.entrypoints.drafts.delete_draft_for_resource(ep_id)
client.entrypoints.drafts.delete_by_draft_id(draft_id)



#BROKEN???
client.plugins.get_all()
client.entrypoints.add_plugins_by_entrypoint_id(ep_id, [plugin_id2])
client.entrypoints.get_plugins_by_entrypoint_id(ep_id)
client.entrypoints.get_plugins_by_entrypoint_id_plugin_id(ep_id, plugin_id2)
client.entrypoints.delete_plugins_by_entrypoint_id_plugin_id(ep_id, plugin_id2)

client.entrypoints.add_queues_by_entrypoint_id(ep_id,[queue_id])
client.entrypoints.modify_queues_by_entrypoint_id(ep_id,[queue_id])
client.entrypoints.delete_queues_by_entrypoint_id(ep_id)
client.entrypoints.add_queues_by_entrypoint_id(ep_id,[queue_id])
client.entrypoints.delete_queues_by_entrypoint_id_queue_id(ep_id, queue_id)
snap_id = client.entrypoints.get_snapshots_by_entrypoint_id(ep_id)['data'][0]['snapshot']
client.entrypoints.get_snapshots_by_entrypoint_id_snapshot_id(ep_id, snap_id)

client.entrypoints.tags.add(ep_id, [tag_id])
client.entrypoints.tags.modify(ep_id, [tag_id, tag_id2])
client.entrypoints.tags.get(ep_id)
client.entrypoints.tags.delete(ep_id, tag_id)
client.entrypoints.tags.delete_all(ep_id)




client.experiments.add_entrypoints_by_experiment_id(exp_id, [ep_id])
client.experiments.modify_entrypoints_by_experiment_id(exp_id, [ep_id])
client.experiments.get_entrypoints_by_experiment_id(exp_id)
client.experiments.delete_entrypoints_by_experiment_id(exp_id)
client.experiments.add_entrypoints_by_experiment_id(exp_id, [ep_id])
client.entrypoints.add_queues_by_entrypoint_id(ep_id,[queue_id])

# jobs testing #
job_id = client.experiments.create_jobs_by_experiment_id(exp_id, 'job description', queue_id, ep_id, {}, '24h')['id']
job_id = client.jobs.get_all()['data'][0]['id']
snap_id = client.jobs.get_snapshots_by_job_id(job_id)['data'][0]['snapshot']
client.jobs.get_snapshots_by_job_id_snapshot_id(job_id, snap_id)
client.jobs.get_status_by_job_id(job_id)
client.jobs.tags.add(job_id, [tag_id])
client.jobs.tags.modify(job_id, [tag_id, tag_id2])
client.jobs.tags.get(job_id)
client.jobs.tags.delete(job_id, tag_id)
client.jobs.tags.delete_all(job_id)


#job_id = client.experiments.create_jobs_by_experiment_id(exp_id, 'job description', queue_id, ep_id, {}, '24h')['id']

# note - needs entrypoint endpoint

client.experiments.get_jobs_by_experiment_id(exp_id)
client.experiments.get_jobs_by_experiment_id_job_id(exp_id, job_id)
client.experiments.modify_jobs_status_by_experiment_id_job_id(exp_id, job_id, "started")
client.experiments.get_jobs_status_by_experiment_id_job_id(exp_id, job_id)

# after here stopped working?
job_id = client.experiments.create_jobs_by_experiment_id(exp_id, 'job description', queue_id, ep_id, {}, '24h')['id']
snapshot_id = client.experiments.get_snapshots_by_experiment_id(exp_id)['data'][0]['snapshot']
client.experiments.get_snapshots_by_experiment_id_snapshot_id(exp_id, snapshot_id)

client.experiments.tags.add(exp_id, [tag_id])
client.experiments.tags.modify(exp_id, [tag_id, tag_id2])
client.experiments.tags.get(exp_id)
client.experiments.tags.delete_all(exp_id)
client.experiments.tags.add(exp_id, [tag_id])
client.experiments.tags.delete(exp_id, tag_id)

artifact_id = client.artifacts.create(1, "artifact description", job_id, "fancy://my_artifact" + str(int(random.random() * 1000000)) )['id']
client.artifacts.get_all()
client.artifacts.modify_by_id(artifact_id, "artifact corrected")
client.artifacts.get_by_id(artifact_id)
model_id = client.models.create(1, "not a classifier", "i am a  model")['id']
client.models.get_all()
client.models.modify_by_id(model_id, 'new name', 'new desc')
client.models.get_by_id(model_id)
version_id = client.models.create_version_by_model_id(model_id, "model description", artifact_id)['versionNumber']
client.models.get_versions_by_model_id(model_id)
client.models.modify_version_by_model_id_version_id(model_id, version_id, 'oh no i can only change the description')
client.models.get_version_by_model_id_version_id(model_id, version_id)

draft_id = client.models.drafts.create(1, 'estimator draft', 'perhaps a low estimation')['id']
client.models.drafts.modify_by_draft_id(draft_id, 'estimator draft (supervised)', 'removed description')
client.models.drafts.get_all()
client.models.drafts.get_by_draft_id(draft_id)
client.models.drafts.create_draft_for_resource(model_id,  'this is not a draft', 'drafty in here')
client.models.drafts.get_draft_for_resource(model_id)
client.models.drafts.modify_draft_for_resource(model_id,'editing is what drafts are for', 'drafty in here')
client.models.drafts.delete_draft_for_resource(model_id)
client.models.drafts.delete_by_draft_id(draft_id)

client.models.tags.add(model_id, [tag_id])
client.models.tags.modify(model_id, [tag_id, tag_id2])
client.models.tags.get(model_id)
client.models.tags.delete(model_id, tag_id)
client.models.tags.delete_all(model_id)

client.models.delete_by_id(model_id)

client.experiments.delete_entrypoints_by_experiment_id_entrypoint_id(exp_id, ep_id)
client.entrypoints.delete_by_id(ep_id)
client.experiments.delete_by_id(exp_id)
client.jobs.get_all()
client.experiments.delete_jobs_by_experiment_id_job_id(exp_id, job_id)


2024-07-22 11:08:30 [debug    ] Request made.                  data={'username': 'finalman', 'email': 'finalman@dioptra.nccoe.nist.gov', 'password': 'fighteroftheDRAFTman', 'confirmPassword': 'fighteroftheDRAFTman'} method=POST url=http://localhost:20080/api/v1/users/
2024-07-22 11:08:30 [error    ] Error code 400 returned.       data={'username': 'finalman', 'email': 'finalman@dioptra.nccoe.nist.gov', 'password': 'fighteroftheDRAFTman', 'confirmPassword': 'fighteroftheDRAFTman'} method=POST response={"message": "Bad Request - The username on the registration form is not available. Please select another and resubmit."}
 url=http://localhost:20080/api/v1/users/
2024-07-22 11:08:30 [debug    ] Request made.                  data={'username': 'finalman', 'password': 'fighteroftheDRAFTman'} method=POST url=http://localhost:20080/api/v1/auth/login
2024-07-22 11:08:30 [debug    ] Response received.             json={'username': 'finalman', 'status': 'Login successful'}
2024-07-22 11:08:30 [d

{'id': [182], 'status': 'Success'}

In [14]:
delete_all()

2024-07-22 11:08:48 [debug    ] Request made.                  data=None method=GET url=http://localhost:20080/api/v1/experiments/
2024-07-22 11:08:48 [debug    ] Response received.             json={'index': 0, 'isComplete': True, 'totalNumResults': 0, 'first': '/api/v1/experiments?index=0&pageLength=10', 'data': []}
2024-07-22 11:08:48 [debug    ] Request made.                  data=None method=GET url=http://localhost:20080/api/v1/entrypoints/
2024-07-22 11:08:48 [debug    ] Response received.             json={'index': 0, 'isComplete': True, 'totalNumResults': 0, 'first': '/api/v1/entrypoints?index=0&pageLength=10', 'data': []}
2024-07-22 11:08:48 [debug    ] Request made.                  data=None method=GET url=http://localhost:20080/api/v1/jobs/
2024-07-22 11:08:48 [debug    ] Response received.             json={'index': 0, 'isComplete': True, 'totalNumResults': 1, 'first': '/api/v1/jobs?index=0&pageLength=10', 'data': [{'id': 181, 'snapshot': 264, 'group': {'id': 1, 'name': '

In [ ]:
def filter_data_dict(data, *args):
    data.pop('self', None) # don't need self in dictionary
    for p in args:
        data.pop(p, None) # don't need any of these in dictionary
    return data
def create_user(self, myid, username, password, confirm):
    data = filter_data_dict(locals(), 'myid') # just specify the names of parameters that aren't part of data
    return data
    # requests.post( something )
print(create_user(client, 1234, 'u', 'p', 'p'))